In [1]:
import os
import time
import datetime

from meta import *
from utils import *
from model.DataGenerator import DataGenerator
from model.NonLinearFXModel import NonLinearFXModel

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

from sklearn.model_selection import train_test_split

config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 56} ) 
sess = tf.Session(config=config) 
K.set_session(sess)

instrument = GUITAR
fx = DISTORTION
param_setting = fx_param_ids[0]

input_target_file_pairs = get_input_target_fname_pairs(instrument, fx, param_setting)

## PUT AUDIO Loading into a function - do it in data_loader?
n_total_clips = len(input_target_file_pairs)
input_target_pairs =  [0] * n_total_clips
for idx, (f_input, f_target) in enumerate(input_target_file_pairs):
    audio_in = load_audio(os.path.join(params_path[instrument][NO_FX], f_input), idx, n_total_clips, NO_FX)
    audio_target = load_audio(os.path.join(params_path[instrument][fx], f_target), idx, n_total_clips, fx)
    
    input_target_pairs[idx] = (audio_in, audio_target)
    
train_pairs, val_pairs = train_test_split(input_target_pairs,
                                       test_size=params_train.get('val_split'),
                                       random_state=42)


# Generate input target frame pairs
train_data_gen = DataGenerator(train_pairs, floatx=np.float32, batch_size=params_train['batch'], frame_size=params_data['frame_size'], hop_size=params_data['hop_size'], unsupervised=True)
val_data_gen = DataGenerator(val_pairs, floatx=np.float32, batch_size=params_train['batch'], frame_size=params_data['frame_size'], hop_size=params_data['hop_size'], unsupervised=True, train=False)

start = time.time()

now = datetime.datetime.now()
print("Current date and time:")
print(str(now))

tr_loss, val_loss = [0] * params_train.get('n_epochs'), [0] * params_train.get('n_epochs')

model = NonLinearFXModel(params_data=params_data, params_train=params_train, dropout=True, dropout_rate=0.3).build()
opt = Adam(lr=params_train['lr'])
model.compile(optimizer=opt, loss='mae', metrics=['mae'])
model.summary()

early_stop = EarlyStopping(monitor='val_loss', patience=params_train.get('patience'), min_delta=0.001, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1)
callback_list = [early_stop, reduce_lr]

hist = model.fit_generator(train_data_gen,
               steps_per_epoch=params_train.get('n_iterations'), #train_data_gen.n_iterations,
               epochs=params_train.get('n_epochs'),
               validation_data=val_data_gen,
               validation_steps=val_data_gen.n_iterations,
               class_weight=None,
               workers=0,
               verbose=1,
               callbacks=callback_list)

end = time.time()
print('\n=============================Job finalized==========================================================\n')
print('Time elapsed for the job: %7.2f hours' % ((end - start) / 3600.0))
print('\n====================================================================================================\n')

path = '/import/c4dm-04/cv/models/unsupervised1/'

model.save_weights(os.path.join(path, 'model.h5'))
    
# Save the model architecture
with open(os.path.join(path, 'model.json'), 'w') as f:
    f.write(model.to_json())


/homes/cv300/venvs/non-linear-fx/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/homes/cv300/venvs/non-linear-fx/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/homes/cv300/venvs/non-linear-fx/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/homes/cv300/venvs/non-linear-fx/lib/python3.6/site-

loading /import/c4dm-04/cv/fxdataset/guitar/NoFX/G63-57307-1111-20704.wav
loading /import/c4dm-04/cv/fxdataset/guitar/Distortion/G63-57307-4411-37552.wav
loading 2 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G73-53208-1111-20848.wav
loading 2 / 624: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G73-53208-4411-37696.wav
loading 3 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G83-67412-1111-21034.wav
loading 3 / 624: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G83-67412-4411-37882.wav
loading 4 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G61-64600-1111-20658.wav
loading 4 / 624: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G61-64600-4411-37506.wav
loading 5 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G61-67603-1111-20661.wav
loading 5 / 624: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G61-67603-4411-37509.wav
loading 6 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G71-56401-1111-20789.wav
loadin

loading 44 / 624: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G93-56211-4411-38011.wav
loading 45 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G61-58308-1111-20627.wav
loading 45 / 624: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G61-58308-4411-37475.wav
loading 46 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G61-63504-1111-20649.wav
loading 46 / 624: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G61-63504-4411-37497.wav
loading 47 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G71-67508-1111-20809.wav
loading 47 / 624: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G71-67508-4411-37657.wav
loading 48 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G83-57307-1111-21016.wav
loading 48 / 624: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G83-57307-4411-37864.wav
loading 49 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G83-71607-1111-21055.wav
loading 49 / 624: distortion /import/c4dm-04/cv/fx

loading 88 / 624: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G93-58403-4411-38029.wav
loading 89 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G93-67603-1111-21207.wav
loading 89 / 624: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G93-67603-4411-38055.wav
loading 90 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G83-52302-1111-21011.wav
loading 90 / 624: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G83-52302-4411-37859.wav
loading 91 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G91-60405-1111-21105.wav
loading 91 / 624: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G91-60405-4411-37953.wav
loading 92 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G93-68604-1111-21208.wav
loading 92 / 624: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G93-68604-4411-38056.wav
loading 93 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G71-74610-1111-20824.wav
loading 93 / 624: distortion /import/c4dm-04/cv/fx

loading 131 / 624: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G83-61406-4411-37876.wav
loading 132 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G73-60501-1111-20880.wav
loading 132 / 624: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G73-60501-4411-37728.wav
loading 133 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G61-69605-1111-20663.wav
loading 133 / 624: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G61-69605-4411-37511.wav
loading 134 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G81-59309-1111-20940.wav
loading 134 / 624: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G81-59309-4411-37788.wav
loading 135 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G71-75611-1111-20825.wav
loading 135 / 624: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G71-75611-4411-37673.wav
loading 136 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G61-71607-1111-20665.wav
loading 136 / 624: distortion /import/c4

...

loading 606 / 624: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G81-72608-4411-37826.wav
loading 607 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G93-67412-1111-21190.wav
loading 607 / 624: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G93-67412-4411-38038.wav
loading 608 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G81-47202-1111-20920.wav
loading 608 / 624: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G81-47202-4411-37768.wav
loading 609 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G81-75611-1111-20981.wav
loading 609 / 624: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G81-75611-4411-37829.wav
loading 610 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G71-56306-1111-20781.wav
loading 610 / 624: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G71-56306-4411-37629.wav
loading 611 / 624: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G61-75611-1111-20669.wav
loading 611 / 624: distortion /import/c4

index 201
index start 80428
index 202
index start 80912
index 203
index start 81308
index 204
index start 81688
index 205
index start 82068
index 206
index start 82552
index 207
index start 82932
index 208
index start 83304
index 209
index start 83692
index 210
index start 84088
index 211
index start 84484
index 212
index start 84864
index 213
index start 85236
index 214
index start 85688
index 215
index start 86060
index 216
index start 86440
index 217
index start 86924
index 218
index start 87312
index 219
index start 87740
index 220
index start 88112
index 221
index start 88484
index 222
index start 88872
index 223
index start 89276
index 224
index start 89664
index 225
index start 90084
index 226
index start 90488
index 227
index start 90868
index 228
index start 91240
index 229
index start 91612
index 230
index start 91984
index 231
index start 92468
index 232
index start 92848
index 233
index start 93220
index 234
index start 93616
index 235
index start 93996
index 236
index star

index 497
index start 198572
index 498
index start 198952
index 499
index start 199436
index 500
index start 199872
index 501
index start 200268
index 502
index start 200648
index 503
index start 201132
index 504
index start 201520
index 505
index start 201900
index 506
index start 202280
index 507
index start 202724
index 508
index start 203120
index 509
index start 203492
index 510
index start 203880
index 511
index start 204268
index 512
index start 204752
index 513
index start 205132
index 514
index start 205504
index 515
index start 205884
index 516
index start 206368
index 517
index start 206748
index 518
index start 207120
index 519
index start 207492
index 520
index start 207880
index 521
index start 208260
index 522
index start 208696
index 523
index start 209068
index 524
index start 209456
index 525
index start 209836
index 526
index start 210208
index 527
index start 210652
index 528
index start 211032
index 529
index start 211412
Loaded 211784 frames
Loading audio frames
i

Epoch 1/100
 801/1000 [=======================>......] - ETA: 56s - loss: 0.1289 - mean_absolute_error: 0.0929

KeyboardInterrupt: 

In [23]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8080862165712774587
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 7201400544938720122
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 11027665058180680305
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 12047777792
locality {
  bus_id: 2
  numa_node: 1
  links {
  }
}
incarnation: 2260112196339021672
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:85:00.0, compute capability: 3.7"
]


In [6]:
!export CUDA_VISIBLE_DEVICES=2

# Compile model

In [1]:
import os
import time
import datetime

from meta import *
from utils import *
from model.DataGenerator import DataGenerator
from model.NonLinearFXModel import NonLinearFXModel

import tensorflow as tf
from tensorflow.keras import backend
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

from sklearn.model_selection import train_test_split

from keras import backend as K
K.clear_session()

model = NonLinearFXModel(params_data=params_data, params_train=params_train, dropout=True, dropout_rate=0.3, dnn=True).build()
opt = Adam(lr=params_train['lr'])
model.compile(optimizer=opt, loss='mae')
model.summary()

/homes/cv300/venvs/non-linear-fx/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/homes/cv300/venvs/non-linear-fx/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/homes/cv300/venvs/non-linear-fx/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/homes/cv300/venvs/non-linear-fx/lib/python3.6/site-

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_frame (InputLayer)        (None, 1024, 1)      0                                            
__________________________________________________________________________________________________
conv1 (Conv1D)                  (None, 1024, 128)    8320        input_frame[0][0]                
__________________________________________________________________________________________________
abs_activation (Lambda)         (None, 1024, 128)    0           conv1[0][0]                      
__________________________________________________________________________________________________
conv1d_local_1 (Conv1DLocal)    (None, 1024, 128)    16512       abs_activation[0][0]             
__________________________________________________________________________________________________
batch_norm

# Load weights into trained unsupervised model and add DNN

In [6]:
import os
import time
import datetime

from meta import *
from utils import *
from model.DataGenerator import DataGenerator
from model.NonLinearFXModel import NonLinearFXModel

import tensorflow as tf
from tensorflow.keras import backend
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

from sklearn.model_selection import train_test_split

import tensorflow as tf

from tensorflow.keras.models import load_model, model_from_json
from tensorflow.keras.layers import Input

from tensorflow.keras.models import Model

out_path = '/import/c4dm-04/cv/models/guitar_distortion_1_2/'

dropout = True
nlfx = NonLinearFXModel(params_data=params_data, params_train=params_train, dropout=dropout, dropout_rate=0.3)
model = nlfx.build()
model.load_weights(os.path.join(out_path, 'unsupervised_model.h5'))

model.summary()

def get_layer_index(model, layer_name):
    index = None
    for idx, layer in enumerate(model.layers):
        if layer.name == layer_name:
            index = idx
            break
    return index

layers = [l for l in model.layers]
idx = get_layer_index(model, 'max_pool')
latent_dnn = nlfx.latent_dnn(model.layers[idx].output)
x = latent_dnn
for i in range(idx + 1, len(layers)):
    if layers[i].name.startswith('multiply'): # handle case where we hit multiply layer 
        x = layers[i]([x, model.get_layer('conv1').output])
    else:
        x = layers[i](x)

# Final touch
result_model = Model(inputs=layers[0].input, outputs=x)
result_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_frame (InputLayer)        (None, 1024, 1)      0                                            
__________________________________________________________________________________________________
conv1 (Conv1D)                  (None, 1024, 128)    8320        input_frame[0][0]                
__________________________________________________________________________________________________
abs_activation (Lambda)         (None, 1024, 128)    0           conv1[0][0]                      
__________________________________________________________________________________________________
conv1d_local_7 (Conv1DLocal)    (None, 1024, 128)    16512       abs_activation[0][0]             
__________________________________________________________________________________________________
batch_norm

In [4]:
import os
import time
import datetime

from meta import *
from utils import *
from model.DataGenerator import DataGenerator
from model.NonLinearFXModel import NonLinearFXModel

import tensorflow as tf
from tensorflow.keras import backend
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

from sklearn.model_selection import train_test_split

import tensorflow as tf

from tensorflow.keras.models import load_model, model_from_json
from tensorflow.keras.layers import Input

from tensorflow.keras.models import Model

out_path = '/import/c4dm-04/cv/models/guitar_distortion_1_2/'

dropout = True
nlfx = NonLinearFXModel(params_data=params_data, params_train=params_train, dropout=dropout, dropout_rate=0.3)
model = nlfx.build()
opt = Adam(lr=params_train['lr'])
model.compile(optimizer=opt, loss='mae', metrics=['mae'])
model.load_weights(os.path.join(out_path, 'unsupervised_model.h5'))

model.summary()

def get_layer_index(model, layer_name):
    index = None
    for idx, layer in enumerate(model.layers):
        if layer.name == layer_name:
            index = idx
            break
    return index

layers = [l for l in model.layers]

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_frame (InputLayer)        (None, 1024, 1)      0                                            
__________________________________________________________________________________________________
conv1 (Conv1D)                  (None, 1024, 128)    8320        input_frame[0][0]                
__________________________________________________________________________________________________
abs_activation (Lambda)         (None, 1024, 128)    0           conv1[0][0]                      
__________________________________________________________________________________________________
conv1d_local_7 (Conv1DLocal)    (None, 1024, 128)    16512       abs_activation[0][0]             
__________________________________________________________________________________________________
batch_norm

In [13]:
layers[7].name

'multiply_9'

# Test load audio

In [1]:
import pandas as pd
from utils import *
from meta import *

# input_target_file_pairs = get_input_target_fname_pairs(GUITAR, DISTORTION, fx_param_ids[0])

# audio_in = load_audio(os.path.join(params_path[GUITAR][NO_FX], input_target_file_pairs[462][0]))
# audio_target = load_audio(os.path.join(params_path[GUITAR][DISTORTION], input_target_file_pairs[462][1]))
# input_target_pairs
df = pd.read_csv('/import/c4dm-04/cv/models/guitar_distortion_1_2/test_data.csv')

import IPython.display as ipd


i = '/import/c4dm-04/cv/fxdataset/guitar/NoFX/{0}'.format(df['input_file'][0])
t = '/import/c4dm-04/cv/fxdataset/guitar/Distortion/{0}'.format(df['target_file'][0])
ipd.Audio(i) # load a local WAV file
ipd.Audio(t) # load a local WAV file

loading /import/c4dm-04/cv/fxdataset/guitar/NoFX/G73-49109-1111-20836.wav
loading /import/c4dm-04/cv/fxdataset/guitar/Distortion/G73-49109-4411-37684.wav


In [13]:
from utils import load_audio
load_audio('/import/c4dm-04/cv/fxdataset/guitar/Distortion/G91-48203-4411-37925.wav')

import IPython.display as ipd

ipd.Audio('/import/c4dm-04/cv/fxdataset/guitar/Distortion/G91-48203-4411-37925.wav') # load a local WAV file

loading /import/c4dm-04/cv/fxdataset/guitar/Distortion/G91-48203-4411-37925.wav


# Evaluation

In [10]:
from evaluate import load_test_data, load_model
from meta import *
from model.DataGenerator import DataGenerator

model_path = '/import/c4dm-04/cv/models/guitar_distortion_1_2/'
dropout = True

input_target_pairs = load_test_data(model_path, GUITAR, DISTORTION)
# Generate input target frame pairs
data_gen = DataGenerator(input_target_pairs, floatx=np.float32, batch_size=params_train['batch'], frame_size=params_data['frame_size'], hop_size=params_data['hop_size'], unsupervised=False)
input_frames, target_frames = data_gen.get_frames()

model = load_model(model_path, dropout=dropout)





/homes/cv300/venvs/non-linear-fx/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/homes/cv300/venvs/non-linear-fx/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/homes/cv300/venvs/non-linear-fx/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/homes/cv300/venvs/non-linear-fx/lib/python3.6/site-

loading /import/c4dm-04/cv/fxdataset/guitar/NoFX/G91-48203-1111-21077.wav
loading /import/c4dm-04/cv/fxdataset/guitar/Distortion/G91-48203-4411-37925.wav
loading 2 / 125: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G73-62407-1111-20873.wav
loading 2 / 125: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G73-62407-4411-37721.wav
loading 3 / 125: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G73-50300-1111-20853.wav
loading 3 / 125: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G73-50300-4411-37701.wav
loading 4 / 125: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G91-56306-1111-21093.wav
loading 4 / 125: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G91-56306-4411-37941.wav
loading 5 / 125: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G73-41101-1111-20828.wav
loading 5 / 125: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G73-41101-4411-37676.wav
loading 6 / 125: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G73-73609-1111-20901.wav
loadin

loading 45 / 125: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G91-57402-1111-21102.wav
loading 45 / 125: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G91-57402-4411-37950.wav
loading 46 / 125: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G73-59500-1111-20879.wav
loading 46 / 125: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G73-59500-4411-37727.wav
loading 47 / 125: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G63-56401-1111-20711.wav
loading 47 / 125: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G63-56401-4411-37559.wav
loading 48 / 125: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G71-53303-1111-20778.wav
loading 48 / 125: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G71-53303-4411-37626.wav
loading 49 / 125: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G91-55400-1111-21100.wav
loading 49 / 125: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G91-55400-4411-37948.wav
loading 50 / 125: nofx /import/c4dm-04/cv/fxdatase

loading 89 / 125: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G81-67603-1111-20973.wav
loading 89 / 125: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G81-67603-4411-37821.wav
loading 90 / 125: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G63-50110-1111-20681.wav
loading 90 / 125: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G63-50110-4411-37529.wav
loading 91 / 125: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G81-48108-1111-20913.wav
loading 91 / 125: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G81-48108-4411-37761.wav
loading 92 / 125: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G73-70511-1111-20890.wav
loading 92 / 125: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G73-70511-4411-37738.wav
loading 93 / 125: nofx /import/c4dm-04/cv/fxdataset/guitar/NoFX/G73-54304-1111-20857.wav
loading 93 / 125: distortion /import/c4dm-04/cv/fxdataset/guitar/Distortion/G73-54304-4411-37705.wav
loading 94 / 125: nofx /import/c4dm-04/cv/fxdatase

In [7]:
i = input_target_pairs[0][1]
import IPython.display as ipd

ipd.Audio(i.T, rate=16000) # load a local WAV file
# ipd.Audio(t, rate=16000) # load a local WAV file